Passos:
1. Considerar que eu tenho uma tabela dataframe em pandas com W+1 linhas e C+1 colunas de uma determinada moeda_X
2. Separar esse dataframe em 2, um com W linhas e C+1 colunas e o outro com a nossa linha que queremos atacar.
3. Criar uma função de bagging e a partir dela gerar outros $\gamma$ dataframes com n linhas
4. Criar uma função que recebe um dataframe e te devolve um conjunto de K outros, sendo aplicado Purge K-Fold Cross Validation
5. Criar uma função que aplica o modelo de LSTM para cada tabela que ela recebe, treinando, gerando um HP e retornando o valor esperado.

In [1]:
import pandas as pd
import numpy as np
import yfinance as yf

# Parâmetro w ajustável
w = 30      #####     HIPERMARÂMTRO       #####

# Definindo o período dos últimos 5 anos
end_date = pd.to_datetime("today").strftime("%Y-%m-%d")
start_date = (pd.to_datetime("today") - pd.DateOffset(years=5)).strftime("%Y-%m-%d")

# Função para calcular o retorno logarítmico
def log_return(series):
    return np.log(series / series.shift(1))

# Baixando os dados históricos do Ethereum, Bitcoin e Solana
eth_data = yf.download("ETH-USD", start=start_date, end=end_date)
btc_data = yf.download("BTC-USD", start=start_date, end=end_date)
sol_data = yf.download("SOL-USD", start=start_date, end=end_date)

# Calculando o retorno logarítmico diário e fazendo o shift para corrigir as datas
eth_data['Return'] = log_return(eth_data['Close']).shift(-2)
btc_data['Return'] = log_return(btc_data['Close']).shift(-2)
sol_data['Return'] = log_return(sol_data['Close']).shift(-2)

# Sincronizando os dados para garantir que todas as moedas tenham os mesmos dias
combined_data = pd.concat([eth_data['Return'], btc_data['Return'], sol_data['Return']], axis=1, keys=['ETH_Return', 'BTC_Return', 'SOL_Return']).dropna()

# Criando o dataframe final
rows = []
for i in range(w, len(combined_data)):
    eth_returns = combined_data['ETH_Return'].iloc[i-w:i].values
    btc_returns = combined_data['BTC_Return'].iloc[i-w:i].values
    sol_returns = combined_data['SOL_Return'].iloc[i-w:i].values
    eth_age = (combined_data.index[i] - eth_data.index[0]).days
    eth_return_today = combined_data['ETH_Return'].iloc[i]
    
    # Concatenando os dados na linha
    row = np.concatenate([eth_returns, btc_returns, sol_returns, [eth_age], [eth_return_today]])
    rows.append(row)

# Nome das colunas
columns = [f"ETH_Return_Day_{j+1}" for j in range(w)] + \
          [f"BTC_Return_Day_{j+1}" for j in range(w)] + \
          [f"SOL_Return_Day_{j+1}" for j in range(w)] + \
          ['ETH_Age', 'ETH_Return_Today']

# Criando o DataFrame final com a data correta como índice
df_final = pd.DataFrame(rows, columns=columns, index=combined_data.index[w:])

df_final.tail()
df_final.shape


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


(1616, 92)

## Agora temos esse dataframe, de modo reduzido, para a moeda etherium.

### Proxima etapa é uma função que itere sobre esse df, para cada iteração criando outro df que contem os ultimos W dias.

In [2]:
# Criando a função First Transformation
def FirstTransform(df, W):
    vet = []
    Linhas, Colunas = df.shape
    for i in range(Linhas, W+1, -1):
        vet.append(df.iloc[(i-W-1):i])
    return vet

In [3]:
df = df_final
W = 100      ####     HIPER PARAMETRO     #####
vetor  = FirstTransform(df, W)
len(vetor)

1515

## Agora temos vetor, que é um vetor cujos elementos são tabelas com W+1 linhas e C+1 Colunas

### Proxima etapa é separar o alvo e depois realizar um bagging para cada elemento desse vetor, criando $\gamma$ outras tabelas

In [4]:
x = vetor[0].iloc[-1]
x = pd.DataFrame(x)
x = x.T
x


,ETH_Return_Day_1,ETH_Return_Day_2,ETH_Return_Day_3,ETH_Return_Day_4,ETH_Return_Day_5,ETH_Return_Day_6,ETH_Return_Day_7,ETH_Return_Day_8,ETH_Return_Day_9,ETH_Return_Day_10,...,SOL_Return_Day_23,SOL_Return_Day_24,SOL_Return_Day_25,SOL_Return_Day_26,SOL_Return_Day_27,SOL_Return_Day_28,SOL_Return_Day_29,SOL_Return_Day_30,ETH_Age,ETH_Return_Today
2024-10-11,0.03324,-0.009469,-0.041233,-0.011097,0.020024,0.011895,0.039316,0.038335,0.021166,-0.012694,...,-0.003222,0.025805,-0.016876,-0.004157,-0.028858,-0.003071,0.045688,0.007668,1824.0,-0.003578


In [5]:
vetor_alvos = []
for i in range(len(vetor)):
    vetor_alvos.append((pd.DataFrame(vetor[i].iloc[-1])).T)
    vetor[i] = vetor[i].drop(vetor[i].index[-1])

In [6]:
def Bagging(df, n, gamma):
    df_bagged = []
    for i in range(gamma):
        aux = df.sample(n = n, random_state = i)
        aux = aux.sort_index()
        df_bagged.append(aux)
    return df_bagged


In [8]:
n = 80       #####       HIPERPARÂMETRO      #####
gamma = 50      #####       HIPERPARÂMETRO      #####

X_inicial = vetor[0] # depois para backtest basta fazer um for e plotar num grafico sei la
alvo = vetor_alvos[0]
X_bagged = Bagging(X_inicial, n, gamma)
len(X_bagged)


50

## Agora temos o X_bagged, um vetor com gamma elementos, que cada elemento contem n linhas e C colunas

### Proxima etapa é para cada um desses elementos de X_bagged, separar em K tuplas de Training e Validation, em uma proporção 90-10.

In [15]:
val_size = 0.1*len(X_bagged)
for i in range(len(X_bagged)):
    for j in range(int(len(X_bagged) - val_size) + 1):
        validation = X_bagged[i].iloc[j : j + val_size]
        train = X_bagged[i].drop(X_bagged.index[j : j + val_size])
validation

TypeError: cannot do positional indexing on DatetimeIndex with these indexers [5.0] of type float